In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import fits_to_pandas, pandas_to_fits, decode_str_columns, cross_match_data_frames
import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns
from astropy.coordinates import SkyCoord
import astropy.coordinates 
import astropy.units as u
import astroquery

%matplotlib inline

matplotlib settings set


matplotlib settings set


load DESI counterparts

In [7]:
desi = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match_all_candidates.gz_pkl', compression='gzip')
desi

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_desi_id_true_ctp,nway_Separation_EROSITA_DESI,nway_pos_err,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_match_flag,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,nway_closest_is_psf
20287,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.110237,132.352465,42.185604,1.670761e-14,1.526396e+20,9011_606779_136,NaN,27.409272,2.982013,0.958140,3.150437e-20,-99.000000,-99.000000,0.307732,0,14,0,False,False,9011,136,606779,154.750219,56.506410,0.218289,0.259747,0.961538,0.669780,3.749257,-2.033533,86.168680,1064.01840,341.324500,68.936470,4.096830,1.032611,0.001391,0.000018,24.056936,23.899303,22.506605,22.929703,21.061779,NaN,17.661356,7.120420,4.798818,7.983458,1.355677,3.809900,-0.075839,0.361240,PSF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0,58194.388402,57784.424786,2018-03-17 09:19:17.912,2017-01-31 10:11:41.491,True
20286,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.110237,132.352465,42.185604,1.670761e-14,1.526396e+20,9011_606779_180,NaN,26.037153,2.982013,0.958140,4.687583e-19,-99.000000,-99.000000,-99.000000,0,14,0,False,False,9011,180,606779,154.755248,56.511001,0.268750,0.578808,0.497439,-2.069496,1.085520,52.015780,-120.210266,475.90520,104.296770,25.880224,3.847811,0.972643,0.001309,0.000017,23.831264,23.029425,23.222218,NaN,22.407553,18.208946,NaN,5.862843,5.911125,2.530604,-4.059490,1.070568,1.882056,-0.498852,REX,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0,58194.388402,57784.424786,2018-03-17 09:19:17.912,2017-01-31 10:11:41.491,True
20285,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,

In [15]:
print(desi.srcname_fin.nunique())
print(desi.query('nway_n_match_flag_2==0').srcname_fin.nunique())
print(desi.query('nway_n_match_flag_2==1').srcname_fin.nunique())
print(desi.query('nway_n_match_flag_2>=2').srcname_fin.nunique())

9215
8462
664
89


In [16]:
print(desi.query("`flux_05-20`>1e-14").srcname_fin.nunique())
print(desi.query("`flux_05-20`>1e-14").query('nway_n_match_flag_2==0').srcname_fin.nunique())
print(desi.query("`flux_05-20`>1e-14").query('nway_n_match_flag_2==1').srcname_fin.nunique())
print(desi.query("`flux_05-20`>1e-14").query('nway_n_match_flag_2>=2').srcname_fin.nunique())

2508
2424
75
9


In [3]:
desi = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match.gz_pkl', compression='gzip')
desi.reset_index(drop=True, inplace=True)
erosita_columns = list(desi.columns.values)
desi.head(10)

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_desi_id_true_ctp,nway_Separation_EROSITA_DESI,nway_pos_err,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_match_flag,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,nway_closest_is_psf
0,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.110237,132.352465,42.185604,1.670761e-14,1.526396e+20,9011_606779_231,NaN,5.167984,2.982013,9.581405e-01,0.992349,0.949328,0.923767,0.845811,1,14,0,False,True,9011,231,606779,154.762154,56.504538,3.362170,4.618722,5.989522,16.031805,20.984790,50.180990,-3.493270,435.342250,248.158630,104.400180,3.761635,0.939089,0.001432,0.000018,21.087423,20.774010,20.520338,19.482046,19.191862,18.247932,NaN,70.151200,72.758970,61.198788,31.093582,20.335644,1.898610,-0.014626,PSF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0,58194.388402,57784.424786,2018-03-17 09:19:17.912,2017-01-31 10:11:41.491,True
1,SRGe J101903.4+563325,154.764302,56.556994,154.897407,50.039645,12.232697,16.660557,2.758328e-14,6.850312e-15,34.209091,8.495835,1133.034668,0.0,0.0,0.0,4806,3902,154.766311,56.555705,5.407337,163.025894,0.000029,6.268413e+08,6.270701e+08,0.0,0.0,0.030192,0.007498,6.562579,154.764302,56.556994,-1,-1.000000,0,-1,NaN,-1.0,-1,SDSS J101904.67+563333.2,1.464000,QSO,13.943999,1,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,68265812198,5.407337,4.373272,6.601858,10.704663,132.318989,42.230639,2.122641e-14,1.507868e+20,9011_606779_284,NaN,12.929691,3.935945,2.397386e-01,0.843921,0.957462,0.923375,0.551305,1,9,0,False,False,9011,284,606779,154.769507,56.559155,4.110681,6.741001,7.775545,18.952187,29.332830,65.296720,371.384800,259.819270,99.752320,72.978060,3.654050,0.866400,0.001278,0.000015,20.872555,20.365770,20.238258,19.300547,18.828358,17.962076,16.075178,66.259700,67.326480,66.424300,36.228188,27.303180,2.334288,1.453649,PSF,-0.009351,1.285011,-0.184517,0.341205,-2.763030,0.601020,G2,853371518808768256,58194.388402,57784.424786,2018-03-17 09:19:17.912,2017-01-31 10:11:41.491,True
2,SRGe J101911.0+562639,154.795822,56.444182,155.033709,50.112976,7.855049,18.953989,1.654359e-14,3.709296e-15,40.391277,9.056271,2230.516357,0.0,0.0,0.0,4530,3655,154.797830,56.442893,3.409649,142.069946,0.000015,6.268413e+08,6.270701e+08,0.0,0.0,0.018108,0.004060,12.856621,154.795822,56.444182,-1,-1.000000,0

# Gaia cross match with epoch correction

In [4]:
gaia = pd.read_csv(data_path+'gaia_dr3_astroph_4-result.csv')

In [5]:

def gaia_coord_correction(df: pd.DataFrame,
                          s_n_threshold: int,
                          epochs_dict: dict = {'desi_epoch': 2000, 'ero_epoch': 2019.8},
                          replace_original_coords: bool = False) -> pd.DataFrame:
    """
    source: https://github.com/mbelveder/luminosity_LH/blob/ab9529c3ee2067a4060c1c3175f8a509ad1c6605/lh_functions.py#L325
    Corrects Gaia coordinates for the proper motion.
    ra  + 1. / 3600e3 * pmra_gaia  * (epoch - 2015.5) / cos(radians(dec)) as ra_gaia,
    dec + 1. / 3600e3 * pmdec_gaia * (epoch - 2015.5) as dec_gaia
    ref: https://www.cosmos.esa.int/web/gaia-users/archive/combine-with-other-data
    Args:
        df (pd.DataFrame): Gaia catalogue.
        epoch (float): Epoch for which coordinates will be propagated.
        s_n_threshold (int): S/N threshold for pmra and pmdec.
        replace_original_coords (bool): If True, replaces original Gaia coordinates with corrected ones.
    Returns:
        pd.DataFrame: Gaia catalogue with corrected coordinates.
    """
    gaia_epoch = 2016

    print(f'Gaia epoch: {gaia_epoch}')

    # Select only reliable pmra and pmdec (allows avoiding correction on noise)
    pmra_sn = df['pmra'].abs() / df['pmra_error']
    df['reliable_pmra'] = np.where(pmra_sn > s_n_threshold, df['pmra'], np.nan)

    pmdec_sn = df['pmdec'].abs() / df['pmdec_error']
    df['reliable_pmdec'] = np.where(pmdec_sn > s_n_threshold, df['pmdec'], np.nan)

    for postfix, epoch in epochs_dict.items():

        print()
        print(f'Target epoch: {epoch}')

        ra_prop_name = f'ra_{postfix}'
        dec_prop_name = f'dec_{postfix}'

        # Positional corrections for objrcts with reliable proper motion measurements
        df[ra_prop_name] = df['ra']  + df['reliable_pmra'] / 3600e3  * (epoch - gaia_epoch)\
                                    / np.cos(np.radians(df['dec']))
        df[dec_prop_name] = df['dec'] + df['reliable_pmdec'] / 3600e3 * (epoch - gaia_epoch)

        # Fill all NaN with original coordinates
        df[ra_prop_name] = df[ra_prop_name].fillna(df['ra'])
        df[dec_prop_name] = df[dec_prop_name].fillna(df['dec'])


        print(f'Column names: {ra_prop_name}, {dec_prop_name}, sep_{postfix}')

    if replace_original_coords:
        df['ra'] = df[ra_prop_name]
        df['dec'] = df[dec_prop_name]

    return df

#gaia = gaia_coord_correction(gaia, s_n_threshold=3, epochs_dict={'desi_epoch': 2000})
gaia = gaia

In [6]:
# desi_gaia = cross_match_data_frames(desi, gaia, 
#                                 colname_ra1='desi_ra',
#                                 colname_dec1='desi_dec',
#                                 colname_ra2='ra_desi_epoch',
#                                 colname_dec2='dec_desi_epoch',
#                                 match_radius = 1,
#                                 df_prefix = 'GAIA',
#                                 closest=True)
# len(desi_gaia)


desi_gaia = cross_match_data_frames(desi, gaia, 
                                colname_ra1='desi_ra',
                                colname_dec1='desi_dec',
                                colname_ra2='ra',
                                colname_dec2='dec',
                                match_radius = 1,
                                df_prefix = 'GAIA',
                                closest=True)
len(desi_gaia)

cross-match radius 1 arcsec
total matches: 2890 out of 9215 x 129743
total closest matches: 2879


2879

# XMATCH

http://cdsxmatch.u-strasbg.fr

## with SDSS DR16

In [7]:
# sdssq = pd.read_pickle(data_path+'SDSS_DR16Q_v4_LH.pkl')
# sdssq.SDSS_NAME = sdssq.SDSS_NAME.str.decode('utf-8')
# sdssq.SDSS_NAME =  'SDSS J' + sdssq.SDSS_NAME.astype(str)
# desi_sdss = cross_match_data_frames(desi, sdssq, 
#                                 colname_ra1='desi_ra',
#                                 colname_dec1='desi_dec',
#                                 colname_ra2='RA',
#                                 colname_dec2='DEC',
#                                 match_radius = 1,
#                                 df_prefix = 'SDSS',
#                                 closest=True)
# desi_sdss


https://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=V/154

In [ ]:
desi[['nway_desi_id', 'desi_ra', 'desi_dec']].to_csv(data_path+'desi_LH_coords.csv', index=False)

In [8]:
##TODO ADD SDSS DR16 FROM XMATCH
xmatch_sdss = pd.read_csv(data_path+'desi_LH_coords_sdssdr16.csv', dtype={'objID': 'str'})
print(len(xmatch_sdss), 'sdss sources after cross-match')
xmatch_sdss = xmatch_sdss.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
xmatch_sdss = xmatch_sdss.query('~spCl.isna()') #remove objects without classification
print(len(xmatch_sdss), 'sdss sources the classification')
xmatch_sdss.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
new_cols = ['SDSS_'+x  if 'desi' not in x  else x  for x in xmatch_sdss.columns]
xmatch_sdss.columns = new_cols
print(len(xmatch_sdss), 'sdss sources the closest filter')
xmatch_sdss
xmatch_sdss.SDSS_spCl.value_counts()
#add SDSS to SDSS_objID
xmatch_sdss['SDSS_NAME']   = xmatch_sdss.SDSS_SDSS16
desi_sdss = desi.merge(xmatch_sdss, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')

20635 sdss sources after cross-match
2347 sdss sources the classification
2347 sdss sources the closest filter


In [13]:
desi_sdss.SDSS_spCl.value_counts()

QSO       2101
GALAXY     238
STAR         8
Name: SDSS_spCl, dtype: int64

In [11]:
desi_sdss.groupby('SDSS_spCl')['SDSS_subCl'].value_counts()

SDSS_spCl  SDSS_subCl 
GALAXY     STARFORMING      33
           AGN              16
           STARBURST        16
           BROADLINE         8
QSO        BROADLINE      1613
           STARBURST        60
           AGN               6
           STARFORMING       3
STAR       M5                2
           O9.5Iae           2
           F9                1
           K5                1
           M4                1
           O8e               1
Name: SDSS_subCl, dtype: int64

## with SIMBAD

In [14]:
# xmatch_simbad = pd.read_csv(data_path+'desi_LH_coords_simbad.csv')
# xmatch_simbad = xmatch_simbad.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
# #xmatch_simbad.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
# new_cols = ['XMATCH_Simbad_'+x  if 'desi' not in x  else x  for x in xmatch_simbad.columns]
# xmatch_simbad.columns = new_cols
# xmatch_simbad

## with millquas

https://heasarc.gsfc.nasa.gov/W3Browse/all/milliquas.html


https://quasars.org/Milliquas-ReadMe.txt

In [98]:
milq = pd.read_csv(data_path+'milliquas_LH.csv', sep=',')
milq.NAME = milq.NAME.str.strip()
milq


,RA,DEC,NAME,TYPE,RMAG,BMAG,COMMENT,R,B,Z,CITE,ZCITE,RXPCT,QPCT,XNAME,RNAME,LOBE1,LOBE2
0,154.764290,58.221086,SDSS J101903.42+581315.9,GR2,21.96,23.74,g,1,1,0.831,DR16,DR16,97.0,3,,FIRST J101903.4+581314,FIRST J101902.8+581302,FIRST J101904.8+581337
1,154.764559,56.914829,SDSS J101903.49+565453.3,Q,20.88,21.02,g,-,-,1.939,DR16Q,DR16Q,NaN,-32768,,,,
2,154.768121,55.474116,SDSS J101904.34+552826.8,Q,21.60,21.70,g,-,-,0.980,DR16Q,DR16Q,NaN,-32768,,,,
3,154.769509,56.559167,SDSS J101904.68+563332.9,Q,20.25,20.67,gG,-,-,1.464,DR16Q,DR16Q,NaN,-32768,,,,
4,154.769562,58.311203,SDSS J101904.69+581840.3,Q,21.10,21.41,g,-,-,1.205,DR16Q,DR16Q,NaN,-32768,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7722,167.079417,54.698561,SDSS J110819.06+544154.8,Q,21.59,22.18,g,-,-,1.571,DR16Q,DR16Q,NaN,-32768,,,,
7723,167.082690,56.258243,PGC 2530909,GR,15.75,18.20,p+,1,1,0.138,PGC,DR16,97.0,1,,FIRST J110819.9+561528,,
7724,167.082922,55.969132,SDSS J110819.90+555808.8,Q,21.06,21.31,g,-,-,0.929,DR16Q,DR16,NaN,-32768,,,,
7725,167.083031,55.387843,SDSS J110819.92+552316.3,Q,20.38,20.43,gG,1,1,1.832,DR16Q,DR16,NaN,-32768,,,,


In [99]:
desi_milq = cross_match_data_frames(desi, milq, 
                                colname_ra1='desi_ra',
                                colname_dec1='desi_dec',
                                colname_ra2='RA',
                                colname_dec2='DEC',
                                match_radius = 2,
                                df_prefix = 'MILQ',
                                closest=True)

desi_milq.MILQ_TYPE = desi_milq.MILQ_TYPE.str.strip()
desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.str.replace('R', '')
desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.str.replace('X', '')
desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.str.replace('2', '')
type_dict_milq = {
    'Q': 'QSO',
    'A': 'AGN',
    'B': 'BLAZAR',
    'L': 'QSO-LENSED',
    'K': 'QSO-NL',
    'N': 'AGN-NL',
    'G': 'GALAXY',
    'S': 'STAR',
    'C': 'CV',
    'M': 'MOVING-OBJECT',
    '': 'RADIO-SOURCE',
}


desi_milq['MILQ_TYPE'] = desi_milq.MILQ_TYPE.map(type_dict_milq)


cross-match radius 2 arcsec
total matches: 2738 out of 9215 x 7727
total closest matches: 2738


In [17]:
# #ADD XMATCH with MILQ
# # f = astropy.io.fits.open(data_path+'milliquas.fits')
# # mask_ra = (f[1].data['RA'] > desi[['RA_fin', 'DEC_fin']].min().RA_fin) & (f[1].data['RA'] < desi[['RA_fin', 'DEC_fin']].max().RA_fin)
# # mask_dec = (f[1].data['DEC'] > desi[['RA_fin', 'DEC_fin']].min().DEC_fin) & (f[1].data['DEC'] < desi[['RA_fin', 'DEC_fin']].max().DEC_fin)
# # mask = mask_ra & mask_dec
# # data_tmp = f[1].data[mask]
# # milq = pd.DataFrame(data_tmp)
# # milq.to_csv(data_path+'milliquas_LH.csv', index=False)
# #milq = pd.read_csv(data_path+'milliquas_LH.csv')
# xmatch_milq = pd.read_csv(data_path+'desi_LH_coords_milliquas.csv')
# xmatch_milq = xmatch_milq.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
# print('milq LH match (2arcsec)', len(xmatch_milq))

# xmatch_milq.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
# new_cols = ['MILQ_'+x  if 'desi' not in x  else x  for x in xmatch_milq.columns]
# xmatch_milq.columns = new_cols
# print('milq LH match (2arcsec) - closest', len(xmatch_milq))

# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('R', '')
# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('X', '')
# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('2', '')
# type_dict_milq = {
#     'Q': 'QSO',
#     'A': 'AGN',
#     'B': 'BLAZAR',
#     'L': 'QSO-LENSED',
#     'K': 'QSO-NL',
#     'N': 'AGN-NL',
#     'G': 'GALAXY',
#     'S': 'STAR',
#     'C': 'CV',
#     'M': 'MOVING-OBJECT',
#     '': 'RADIO-SOURCE',
# }
# xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.map(type_dict_milq)
# desi_milq = desi.merge(xmatch_milq, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')

# xmatch_milq



### compare SDSS and MILQ

In [100]:
desi_sdss_milq = desi_sdss.merge(desi_milq, on=erosita_columns, how='right')
#test_cols = ['SDSS_angDist', 'MILQ_angDist','SDSS_SDSS16', 'SDSS_spCl', 'SDSS_subCl',  'MILQ_NAME', 'MILQ_TYPE', 'SDSS_zsp', 'MILQ_Z']

In [101]:
tmp_df = desi_sdss_milq.query("~SDSS_NAME.isna() & ~MILQ_NAME.isna() & (MILQ_NAME == SDSS_NAME)  ")#[test_cols]
print('sdss -  milq complete match', len(tmp_df))
tmp_df.sample(5)

sdss -  milq complete match 1818


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_desi_id_true_ctp,nway_Separation_EROSITA_DESI,nway_pos_err,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,SDSS_angDist,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD,SDSS_NAME,MILQ_sep,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2
335,SRGe J103141.9+583101,157.924539,58.516962,150.877174,50.344826,5.122559,57.771095,1.416460e-14,2.400374e-15,50.946781,8.633594,3285.943359,0.0,0.0,0.0,2653,2114,157.926548,58.515673,2.126717,64.711105,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.015504,0.002627,16.128658,157.924539,58.516962,855108407877798912,6.559351,1,0,19.788912,-1.0,0,SDSS J103141.62+583059.3,1.400491,QSO,6.558104,1,0,0,1,16.040001,14.918000,12.680,21.5,18.1,2.3,1,1,1,1237653616396599534,9220034594266697728,6.528180,60694721320,2.126717,1.831350,2.764591,4.482680,132.982556,44.777093,1.090021e-14,7.765359e+19,9011_613007_2576,NaN,2.573620,1.648215,0.989966,1.000000,0.959000,0.930576,0.467800,10,0,True,True,9011,2576,613007,157.923461,58.516522,6.145496,11.602101,9.958595,35.484165,52.887320,62.975697,-138.13217,285.37600,132.46411,143.232070,3.241173,0.804330,0.001459,0.000018,20.494701,19.815819,19.991730,18.622972,18.190430,18.001814,NaN,103.81634,133.532140,119.184074,63.883053,47.431710,2.405254,-0.582005,-0.291644,4.361084,0.068277,3.813668,0.332393,1.855253,G2,855108407877798912,0.137495,1237653616396599534,157.923443,58.516485,0.061,0.054,0.0,1.0,6.0,1.0,0.054,0.061,21.882,21.831,21.231,21.005,21.198,0.174,0.070,0.068,0.082,0.402,1.40539,0.00114,0.0,QSO,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,3.0,SDSS J103141.62+583059.3,8189-57448-0146,51875.4515,SDSS J103141.62+583059.3,0.005305,157.923460,58.516520,SDSS J103141.62+583059.3,QSO,21.28,21.87,gG,-,-,1.405,DR16Q,DR16,NaN,-32768,,,,
2715,SRGe J110541.9+581855,166.424775,58.315400,146.234294,53.765435,9.856372,9.758303,1.939187e-14,6.856689e-15,14.481753,5.120543,682.258606,0.0,0.0,0.0,8246,7100,166.426784,58.314110,4.327298,77.032669,0.000015,6.268413e+08,6.270701e+08,0.0,0.0,0.021226,0.007505,5.708407,166.424775,58.315400,857342409347995776,9.967412,1,0,18.954660,-1.0,0,[VV2006] J110542.3+581900,1.613930,QSO,9.968176,1,1,0,0,15.581000,15.324000,12.680,26.9,13.2,0.0,1,2,0,1237655107836904288,0,3.187452,61455245257,4.327298,3.5

In [104]:
tmp_df = desi_sdss_milq.query("~SDSS_NAME.isna() & ~MILQ_NAME.isna() & (MILQ_NAME != SDSS_NAME)  ")
tmp_df['z_diff'] = np.abs(tmp_df['MILQ_Z'] - tmp_df['SDSS_zsp'])
tmp_df = tmp_df.sort_values(by=['z_diff'], ascending=False)
#max_z_diff = (tmp_df.MILQ_Z - tmp_df.SDSS_Z).max()
#print('max z diff', max_z_diff)
print('sdss -  milq un match -', len(tmp_df))
print('sdss -  milq un match - max z diff', tmp_df.z_diff.max())
print('sdss -  milq un match - z diff < 0.02', np.sum(tmp_df.z_diff<0.02))

#print(tmp_df.z_diff.describe())
tmp_df.head(15)

sdss -  milq un match - 371
sdss -  milq un match - max z diff 1.8691799999999998
sdss -  milq un match - z diff < 0.02 361


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_desi_id_true_ctp,nway_Separation_EROSITA_DESI,nway_pos_err,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,SDSS_angDist,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD,SDSS_NAME,MILQ_sep,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2,z_diff
1285,SRGe J104256.0+590427,160.733211,59.074109,148.745782,51.124534,3.699127,246.717377,3.184136e-14,2.942440e-15,148.434570,13.716745,4258.836426,0.0,0.0,0.0,665,545,160.735219,59.072820,1.416959,115.891090,0.000010,6.268413e+08,6.270701e+08,0.0,0.0,0.034853,0.003221,17.266644,160.733211,59.074109,855547834572632960,6.624845,1,1,16.134848,NaN,0,CLANS 18,2.760000,QSO,7.714329,1,1,0,2,13.889000,13.909,12.758,41.9,30.8,-0.4,2,2,1,1237653616397254727,0,6.390868,58618158552,1.416959,1.322463,1.996380,3.237054,134.319242,46.020742,2.450317e-14,7.586878e+19,9011_614515_242,b'9011_614515_242',1.608846,1.190217,0.987404,1.000000,0.884797,0.893998,0.674607,14,0,True,True,9011,242,614515,160.732460,59.074335,0.668765,1.641158,4.425083,41.349422,43.596010,94.285040,62.625553,512.841300,153.916120,54.010777,3.113233,0.886655,0.001291,0.000015,22.912888,21.946005,20.876180,18.457457,18.400541,17.563713,18.008053,15.144859,20.360690,32.520832,72.958400,41.051030,3.387808,0.246488,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0,0.011496,1237653616397254898,160.732461,59.074339,0.030,0.028,0.0,1.0,6.0,1.0,0.028,0.030,21.443,21.097,20.569,20.557,20.099,0.107,0.040,0.037,0.057,0.153,0.89082,0.00003,0.0,GALAXY,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,3.0,SDSS J104255.79+590427.6,7095-56625-0570,51875.4557,SDSS J104255.79+590427.6,0.018346,160.732470,59.074337,CLANS J10429+5904,AGN,20.60,21.11,ga,-,-,2.760,1882,1882,99.0,97,CXOG J104255.7+590428,,4XMM J104255.8+590427,,1.86918
1597,SRGe J104604.6+584424,161.519056,58.739927,148.717363,51.649946,5.804560,53.263432,1.268957e-14,2.087466e-15,60.201984,9.903375,4334.224121,0.0,0.0,0.0,2454,1939,161.521064,58.738638,2.452672,131.308472,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.013890,0.002285,18.390734,161.519056,58.739927,855293400709705088,9.511575,1,0,21.244097,-1.0,0,CLANS 397,1.810000,QSO,9.511712,1,1,0,1,17.410999,15.957,12.295,7.4,7.7,0.5,1,2,0,1237655109446467838,0,8.503062,59860040492,2.452672,2.07517

In [ ]:
# desi_sdss_milq.query("~SDSS_SDSS16.isna() & SDSS_spCl=='GALAXY'")[test_cols]

In [ ]:
# xmatch_sdss_milq.query("~SDSS_SDSS16.isna() & MILQ_NAME.isna() & (MILQ_NAME != SDSS_SDSS16)  ")

In [ ]:
# xmatch_sdss_milq.groupby('MILQ_TYPE')['SDSS_spCl', 'SDSS_subCl'].value_counts()

In [ ]:
# xmatch_sdss_milq.groupby(['SDSS_spCl', 'SDSS_subCl'])['MILQ_TYPE'].value_counts()

# Join all matches

In [116]:
#merge desi_gaia with desi_sdss with desi to create a new desi_gaia_sdss catalog 
#https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
from functools import reduce
data_frames = [desi, desi_gaia, desi_sdss, desi_milq]

desi_matched = reduce(lambda  left,right: pd.merge(left,right,on=erosita_columns,
                                            how='outer'), data_frames).fillna(np.nan)

desi_matched.fillna(np.nan, inplace=True)

desi_matched

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_desi_id_true_ctp,nway_Separation_EROSITA_DESI,nway_pos_err,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,GAIA_sep,GAIA_source_id,GAIA_ra,GAIA_ra_error,GAIA_dec,GAIA_dec_error,GAIA_parallax,GAIA_parallax_error,GAIA_pmra,GAIA_pmra_error,GAIA_pmdec,GAIA_pmdec_error,GAIA_phot_g_mean_flux,GAIA_phot_g_mean_flux_error,GAIA_phot_g_mean_mag,GAIA_phot_bp_mean_flux,GAIA_phot_bp_mean_flux_error,GAIA_phot_bp_mean_mag,GAIA_phot_rp_mean_flux,GAIA_phot_rp_mean_flux_error,GAIA_phot_rp_mean_mag,GAIA_bp_rp,GAIA_classprob_dsc_combmod_quasar,GAIA_classprob_dsc_combmod_galaxy,GAIA_classprob_dsc_combmod_star,GAIA_classprob_dsc_combmod_whitedwarf,GAIA_classprob_dsc_combmod_binarystar,GAIA_spectraltype_esphs,GAIA_distance_gspphot,GAIA_distance_msc,SDSS_angDist,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD,SDSS_NAME,MILQ_sep,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2
0,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.110237,132.352465,42.185604,1.670761e-14,1.526396e+20,9011_606779_231,NaN,5.167984,2.982013,9.581405e-01,0.992349,0.949328,0.923767,0.845811,14,0,False,True,9011,231,606779,154.762154,56.504538,3.362170,4.618722,5.989522,16.031805,20.984790,50.180990,-3.493270,435.342250,248.158630,104.400180,3.761635,0.939089,0.001432,0.000018,21.087423,20.774010,20.520338,19.482046,19.191862,18.247932,NaN,70.151200,72.758970,61.198788,31.093582,20.335644,1.898610,-0.014626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0,0.004428,8.533242e+17,154.762155,3.899419,56.504539,4.484491,NaN,NaN,NaN,NaN,NaN,NaN,79.281657,1.895461,20.939436,12.492804,3.559934,22.596893,51.569261,30.165367,20.466919,2.129974,2.659257e-03,1.216818e-05,9.971745e-01,1.540472e-04,0.000000e+00,NaN,NaN,NaN,0.082520,1237658303814435106,154.762176,56.504558,0

## preliminary classification

GAIA

In [117]:

th = 0.9
qso_mask = desi_matched.eval('(GAIA_classprob_dsc_combmod_quasar > @th) ')
desi_matched.loc[qso_mask, 'class_GAIA_class'] = 'QSO'

galaxy_mask = desi_matched.eval('(GAIA_classprob_dsc_combmod_galaxy > @th) ')
desi_matched.loc[galaxy_mask, 'class_GAIA_class'] = 'GALAXY'



th = 3
star_mask = desi_matched.eval('(GAIA_parallax/GAIA_parallax_error > @th) | (GAIA_pmra/GAIA_pmra_error > @th) | (GAIA_pmdec/GAIA_pmdec_error > @th)')
desi_matched.loc[star_mask, 'class_GAIA_class'] = 'STAR'

#desi_matched.loc[desi_matched.class_GAIA_class.isna() & ~desi_matched.GAIA_DR3Name.isna(), 'class_GAIA_class'] = 'undef'


In [118]:
print('NOTE NANS ARE NOT PRINTER IN .value_counts()')
desi_matched.class_GAIA_class.value_counts()

NOTE NANS ARE NOT PRINTER IN .value_counts()


QSO       1482
STAR       457
GALAXY     220
Name: class_GAIA_class, dtype: int64

SDSS

In [119]:
sdss_class_mask = desi_matched.eval('~SDSS_spCl.isna() ')
desi_matched.loc[sdss_class_mask, 'class_SDSS_class'] = desi_matched.SDSS_spCl[sdss_class_mask]
assert len(desi_sdss.SDSS_spCl) == np.sum(sdss_class_mask), f'SDSS_spCl and class_SDSS_class have different lengths {len(desi_sdss.SDSS_spCl)} {len(desi_matched.class_SDSS_class)}'
#sdss_class_mask = desi_matched.eval('~SDSS_NAME.isna() ')
desi_matched.loc[sdss_class_mask, 'class_SDSS_class'] = desi_matched.SDSS_spCl[sdss_class_mask]

In [120]:
desi_matched.class_SDSS_class.value_counts()

QSO       2101
GALAXY     238
STAR         8
Name: class_SDSS_class, dtype: int64

MILQ

In [121]:
mask_extragal = desi_matched.eval(" MILQ_TYPE in ['QSO', 'AGN', 'AGN-NL', 'QSO-NL', 'BLAZAR']") #GALAXY
desi_matched.loc[mask_extragal, 'class_MILQ_class'] = 'QSO'

mask_extragal = desi_matched.eval(" MILQ_TYPE in ['GALAXY']") #GALAXY
desi_matched.loc[mask_extragal, 'class_MILQ_class'] = 'GALAXY'


## Compare classes

In [122]:
print('both SDSS and MILQ have class and they are the same:')
print(len(desi_matched.query("~class_MILQ_class.isna() & ~class_SDSS_class.isna() & class_MILQ_class==class_SDSS_class")))
print('both SDSS and MILQ have class and they are different:')
print(len(desi_matched.query("~class_MILQ_class.isna() & ~class_SDSS_class.isna() & class_MILQ_class!=class_SDSS_class")))

print('SDSS class available only but not MILQ class:')
print(len(desi_matched.query("class_MILQ_class.isna() & ~class_SDSS_class.isna()")))

print('MILQ class available only but not SDSS class:')
print(len(desi_matched.query("class_SDSS_class.isna() & ~class_MILQ_class.isna()")))

both SDSS and MILQ have class and they are the same:
2130
both SDSS and MILQ have class and they are different:
49
SDSS class available only but not MILQ class:
168
MILQ class available only but not SDSS class:
189


In [123]:
desi_matched.groupby('class_GAIA_class').class_SDSS_class.value_counts()


class_GAIA_class  class_SDSS_class
GALAXY            GALAXY               123
                  QSO                   24
QSO               QSO                 1239
                  GALAXY                 8
                  STAR                   3
STAR              QSO                    8
                  GALAXY                 5
                  STAR                   4
Name: class_SDSS_class, dtype: int64

In [124]:
desi_matched.groupby('class_GAIA_class').class_MILQ_class.value_counts()

class_GAIA_class  class_MILQ_class
GALAXY            QSO                   52
                  GALAXY                35
QSO               QSO                 1260
                  GALAXY                 3
STAR              QSO                   11
                  GALAXY                 1
Name: class_MILQ_class, dtype: int64

When GAIA indicate star but SDSS says QSO

In [ ]:
# #[x for x in desi_gaia_sdss.columns if 'mag' in x]
# mags_cols = ['desi_dered_mag_g', 'SDSS_gmag', 'GAIA_Gmag', 'desi_dered_mag_r', 'SDSS_rmag', 'desi_dered_mag_z', 'SDSS_zmag']
# desi_matched.query('class_SDSS_class=="QSO" & class_GAIA_class=="STAR"')[mags_cols]


## Finalize classes

In [125]:
desi_matched

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_desi_id_true_ctp,nway_Separation_EROSITA_DESI,nway_pos_err,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,GAIA_sep,GAIA_source_id,GAIA_ra,GAIA_ra_error,GAIA_dec,GAIA_dec_error,GAIA_parallax,GAIA_parallax_error,GAIA_pmra,GAIA_pmra_error,GAIA_pmdec,GAIA_pmdec_error,GAIA_phot_g_mean_flux,GAIA_phot_g_mean_flux_error,GAIA_phot_g_mean_mag,GAIA_phot_bp_mean_flux,GAIA_phot_bp_mean_flux_error,GAIA_phot_bp_mean_mag,GAIA_phot_rp_mean_flux,GAIA_phot_rp_mean_flux_error,GAIA_phot_rp_mean_mag,GAIA_bp_rp,GAIA_classprob_dsc_combmod_quasar,GAIA_classprob_dsc_combmod_galaxy,GAIA_classprob_dsc_combmod_star,GAIA_classprob_dsc_combmod_whitedwarf,GAIA_classprob_dsc_combmod_binarystar,GAIA_spectraltype_esphs,GAIA_distance_gspphot,GAIA_distance_msc,SDSS_angDist,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD,SDSS_NAME,MILQ_sep,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2,class_GAIA_class,class_SDSS_class,class_MILQ_class
0,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.110237,132.352465,42.185604,1.670761e-14,1.526396e+20,9011_606779_231,NaN,5.167984,2.982013,9.581405e-01,0.992349,0.949328,0.923767,0.845811,14,0,False,True,9011,231,606779,154.762154,56.504538,3.362170,4.618722,5.989522,16.031805,20.984790,50.180990,-3.493270,435.342250,248.158630,104.400180,3.761635,0.939089,0.001432,0.000018,21.087423,20.774010,20.520338,19.482046,19.191862,18.247932,NaN,70.151200,72.758970,61.198788,31.093582,20.335644,1.898610,-0.014626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0,0.004428,8.533242e+17,154.762155,3.899419,56.504539,4.484491,NaN,NaN,NaN,NaN,NaN,NaN,79.281657,1.895461,20.939436,12.492804,3.559934,22.596893,51.569261,30.165367,20.466919,2.129974,2.659257e-03,1.216818e-05,9.971745e-01,1.540472e-04,0.000000e+00,NaN,NaN,NaN,

In [127]:

def assign_final_class(row):
    if not pd.isna(row.class_MILQ_class):
        return row.class_MILQ_class
    elif not pd.isna(row.class_SDSS_class):
        return row.class_SDSS_class
    elif not pd.isna(row.class_GAIA_class):
        return row.class_GAIA_class
    else:
        return 'unknown'

def assign_final_redshift(row):
    if not pd.isna(row.MILQ_Z):
        return row.MILQ_Z
    elif not pd.isna(row.SDSS_zsp):
        return row.SDSS_zsp
    else:
        return np.nan

desi_matched.loc[:, 'class_final'] = desi_matched.apply(assign_final_class, axis=1)
desi_matched.loc[:, 'redshift_final'] = desi_matched.apply(assign_final_redshift, axis=1)


num_qso = len(desi_matched.query('class_MILQ_class=="QSO"'))
num_qso += len(desi_matched.query('class_SDSS_class=="QSO" & class_MILQ_class.isna()'))
num_qso += len(desi_matched.query('class_SDSS_class.isna() & class_MILQ_class.isna() & class_GAIA_class=="QSO"'))
desi_matched.class_final.value_counts()['QSO'] == num_qso, f'QSO count is wrong, should be {num_qso}, got {desi_matched.class_final.value_counts()["QSO"]}'
desi_matched.class_final.value_counts()

unknown    5964
QSO        2543
STAR        446
GALAXY      262
Name: class_final, dtype: int64

In [128]:
desi_matched.to_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match_xmatch.pkl')